In [ ]:

import torch
from math import factorial, ceil
from pytorch3d.io import load_obj
import timeit
import numpy as np

def M_ijk_torch(f_matrix, v_matrix, max_m, is_watertigh):
    total_num_faces = len(f_matrix)
        
    loop_batch_size = 10000
        
    m_v = torch.zeros([max_m,max_m,max_m],dtype=torch.float32).cuda()
    m_s = torch.zeros([max_m,max_m,max_m],dtype=torch.float32).cuda()
    
    for batch_num in range(ceil(total_num_faces/loop_batch_size)):
        start = batch_num*loop_batch_size
        end = min([(batch_num+1)*loop_batch_size, total_num_faces])
        num_faces = end-start
        
        # Extract coordinates
        ABC = v_matrix[f_matrix[start:end,:]]
        # Calculate Determinants or Cross Products
        if is_watertigh:
            dets = torch.det(ABC).cuda()
        cross = torch.norm(torch.cross(ABC[:,0]-ABC[:,2],ABC[:,1]-ABC[:,2]),p=2,dim = 1).cuda()


        # Allocate Tensors
        M_V_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float32).cuda()
        M_S_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float32).cuda()
        C_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float32).cuda()
        D_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float32).cuda()
        S_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float32).cuda()

        # Calculate C Tensor, parallellized over faces
        for i in range(max_m):
                for j in range(max_m):
                    for k in range(max_m):
                        if (i+j+k)<=max_m:
                            C_tensor[:,i,j,k] = (ABC[:,2][:,0]**i)*(ABC[:,2][:,1]**j)*(ABC[:,2][:,2]**k)*(factorial(i+j+k)/(factorial(i)*factorial(j)*factorial(k)))

        # Calculate D Tensor, parallellized over faces
        for i in range(max_m):
                for j in range(max_m):
                    for k in range(max_m):
                        if (i+j+k)<=max_m:
                            if (i<0) or (j<0) or (k<0):
                                # D_ijk=0
                                pass
                            elif (0==i) and (0==j) and (0==k):
                                # D_ijk = 1
                                D_tensor[:,i,j,k] = 1
                            else:
                                D_tensor[:,i,j,k] = ABC[:,1][:,0]*D_tensor[:,i-1,j,k]+ABC[:,1][:,1]*D_tensor[:,i,j-1,k]+ABC[:,1][:,2]*D_tensor[:,i,j,k-1]+C_tensor[:,i,j,k]


        # Calculate S Tensor, parallellized over faces
        for i in range(max_m):
                for j in range(max_m):
                    for k in range(max_m):
                        if (i+j+k)<=max_m:
                            if (i<0) or (j<0) or (k<0):
                                # S_ijk = 0
                                pass
                            elif (0==i) and (0==j) and (0==k):
                                # S_ijk = 1
                                S_tensor[:,i,j,k] = 1
                            else:
                                S_tensor[:,i,j,k] = ABC[:,0][:,0]*S_tensor[:,i-1,j,k]+ABC[:,0][:,1]*S_tensor[:,i,j-1,k]+ABC[:,0][:,2]*S_tensor[:,i,j,k-1]+D_tensor[:,i,j,k]

        # Calculate M Tensor for Volumetric moments, parallellized over faces
        if is_watertigh:
            for i in range(max_m):
                for j in range(max_m):
                    for k in range(max_m):
                        if (i+j+k)<=max_m:
                            M_V_tensor[:,i,j,k] = ((factorial(i)*factorial(j)*factorial(k))/factorial(i+j+k+3))*(dets[:]*S_tensor[:,i,j,k])

        # Calculate M Tensor for Surface moments, parallellized over faces
        for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        M_S_tensor[:,i,j,k] = ((factorial(i)*factorial(j)*factorial(k))/factorial(i+j+k+2))*(cross[:]*S_tensor[:,i,j,k])

        m_v = m_v + torch.sum(M_V_tensor,0)
        m_s = m_s + torch.sum(M_S_tensor,0)
    
    return (m_v,m_s)
    

def V_C_M(f_matrix,v_matrix,max_m, is_watertigh):
    
    
    # Calculate the moments
    
    if not is_watertigh:
        m = M_ijk_torch(f_matrix,v_matrix,max_m, is_watertigh)
    else:
        # Define face ordering
        order = [0,1,2]

        # Get volume and centroid
        starttime = timeit.default_timer()
        v_c = M_ijk_torch(f_matrix,v_matrix,2, is_watertigh)[0]
        #print("Get volume and centroid is :", timeit.default_timer() - starttime)
        v = torch.pow(v_c[0,0,0], 1/3).cuda()
        c = torch.zeros((1,3)).cuda()
        c[0,0] = v_c[1,0,0]
        c[0,1] = v_c[0,1,0]
        c[0,2] = v_c[0,0,1]

        # Correct face ordering if volume is negative
        if v_c[0,0,0]<0:
            order = [0,2,1]
            v_c = M_ijk_torch(f_matrix[:,torch.LongTensor(order).cuda()],v_matrix,2, is_watertigh)
            #print('Corrected winding')
        
        m = M_ijk_torch(f_matrix[:,torch.LongTensor(order)],(v_matrix - c/v_c[0,0,0])/v,max_m, is_watertigh)
    
    # Return tuple of volume, centroid, and the invariant moments
    return v,c,m[0],m[1]

def VCM_loss(f_matrix_pred,v_matrix_pred,v_gt,c_gt,m_v_gt, m_s_gt, is_watertigh):
    v_pred,c_pred,m_v_pred, m_s_pred = V_C_M(f_matrix_pred,v_matrix_pred, is_watertigh)
    
    v_loss = np.linalg.norm(v_pred - v_gt)
    c_loss = np.linalg.norm(c_pred - c_gt)
    m_v_loss = np.linalg.norm(m_v_pred - m_v_gt)
    m_s_loss = np.linalg.norm(m_s_pred - m_s_gt)
    
    return v_loss, c_loss, m_v_loss, m_s_loss
    
    

    
def compute_moments(f_matrix=None, v_matrix=None, max_m=20, is_watertigh=False):
    assert v_matrix is not None and f_matrix is not None, 'mesh_path or (v_matrix and f_matrix) should be defined'
    v_matrix, f_matrix = v_matrix.cuda(), f_matrix.cuda()
    out = V_C_M(f_matrix, v_matrix, max_m, is_watertigh)
    assert out is not None
    v, c, v_m, s_m = out
    if is_watertigh:
        assert v is not None and v.item() > 0, 'The volume should be greater than zero for watertigh meshes'
    
    assert c is not None
    assert v_m is not None
    assert v_m.shape == (max_m, max_m, max_m)
    assert s_m.shape == (max_m, max_m, max_m)
    assert s_m[0, 0, 0].item() > 0, 'The surface area should be greater than zero'
    
    return v, c, v_m, s_m


if __name__ == "__main__":
    #for _ in range(10):
    #print("Function started")
    v_matrix = [[0, 0, 0], [0, 0, 1], [0, 1 ,0], [0, 1, 1], [1, 0, 0], [1, 0, 1], [1, 1 ,0], [1, 1 ,1]]
    f_matrix = [[0,2,4], [4,2,6],[0,1,3], [0,3,2], [0,5,1], [0,4,5], [1,7,3], [1,5,7], [4,7,5], [4,6,7], [7,6,2],[7,2,3]]
    v_matrix, f_matrix = torch.tensor(v_matrix), torch.tensor(f_matrix)
    #m = torch.load("/gpfs/u/scratch/SCMM/shared/mesh_data/meshrcnn/datasets/shapenet/ShapeNetV1processed/02933112/1055dc4f3f2079f7e6c5cd45aa112726/mesh.pt")
    #m = torch.load("/gpfs/u/scratch/SCMM/shared/mesh_data/meshrcnn/datasets/mesh.pt")
    #vertex_matrix, face_matrix = m['verts'], m['faces']
    print("The face matrix shape", f_matrix.shape)
    print("Vertex and face initialzed")
    max_m = 10
    print("Compute moment")
    out = compute_moments(torch.tensor(f_matrix).cuda(), 
                                     torch.tensor(v_matrix).float().cuda(), 
                                     max_m=max_m, is_watertigh=True)
    assert out is not None
    v, c, v_m, s_m = out
    assert v.sum().item() != 0
    assert c is not None
    assert v_m is not None
    assert v_m.shape == (max_m, max_m, max_m)
    assert s_m.shape == (max_m, max_m, max_m)
    assert s_m.sum().item() != 0, 'The surface moments should never be zero'
    
    test_tensor = torch.tensor(np.zeros((max_m,max_m,max_m))).cuda()
    for i in range(max_m):
        for j in range(max_m):
            for k in range(max_m):
                if (i+j+k)<=max_m:
                    test_tensor[i,j,k] = 1/((i+1)*(j+1)*(k+1))
    
    moments_torch = M_ijk_torch(torch.tensor(f_matrix, dtype=torch.long).cuda(), torch.tensor(v_matrix).float().cuda(), max_m, is_watertigh=True)
    moments_torch_error = torch.linalg.norm(torch.tensor(test_tensor) - moments_torch[0].cuda())
    print('Pytorch implementation error: {}'.format(moments_torch_error))
                                                                                  